# Interface GridSearchCV and Pipelines
The syntax to interface `GridSearchCV` and a pipeline estimator can be tricky. There are two methods, each with its pros and cons. The first method feeds a pipeline estimator into `GridSearchCV` while the second has `GridSearchCV` as a step in the pipeline. Remember, both `Pipeline` and `GridSearchCV` are estimator classes, with the `fit`, `predict`, and `score` methods.

In [1]:
import numpy as np

from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

For this demostration, let's use the California housing dataset and create a `Pipeline` object containing two steps: a feature scaler and a ridge estimator.

In [3]:
# get data and perform train/test split
X = fetch_california_housing()['data']
y = fetch_california_housing()['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# create pipeline estimator
pipe_1 = Pipeline([('scaler', StandardScaler()), ('est', Ridge())])
pipe_1.fit(X_train, y_train)

print (pipe_1.score(X_test, y_test))

0.5943141338604155


## Perform a gridseach on a pipeline
The first method performs a gridsearch on a `Pipeline` estimator. A `Pipeline` object absorbs the attributes and parameters of all the steps, the transformers and estimator, and uses the name of the step as the prefix to the name of the attributes. For example, the ridge regularization parameter is now referred to as `est__alpha` rather than just `alpha`. The prefix is needed distinguish between the hyperparameters among all the transformers and estimator of the pipeline. The `get_params` method returns a dictionary of the pipeline parameters, using the name of the stage and double underscore (dunder) as a prefix.

In [4]:
# print dictionary of pipeline parameters
pipe_1.get_params()

{'memory': None,
 'steps': [('scaler',
   StandardScaler(copy=True, with_mean=True, with_std=True)),
  ('est', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001))],
 'scaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'est': Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'est__alpha': 1.0,
 'est__copy_X': True,
 'est__fit_intercept': True,
 'est__max_iter': None,
 'est__normalize': False,
 'est__random_state': None,
 'est__solver': 'auto',
 'est__tol': 0.001}

To perform a gridsearch on the ridge regularization hyperparameter, the keyword `est__alpha` needs to be used.

In [6]:
# perform hyperparameter tuning on pipeline estimator
h_params = {'est__alpha': np.logspace(-3, 3, 20)}
gs_est = GridSearchCV(pipe_1, h_params, cv=3, n_jobs=2, verbose=1)
gs_est.fit(X_train, y_train)

print (gs_est.score(X_test, y_test))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
0.5941973302098567


[Parallel(n_jobs=2)]: Done  60 out of  60 | elapsed:    1.4s finished


The advantage of feeding a `pipeline` object to `GridSearchCV` is that it allows for tuning all the hyperparameters of the steps in the pipeline, not just the final step, the estimator. For example, if we wanted to explore the affect of modifying the `with_mean` parameter of the scaler transformer, we would use the keyword `scaler__with_mean` since we used the string "scaler" to refer to the scaler transformer when creating our pipeline.

When calling the `fit` method, each training step of the cross-validation scheme will perform the fit and transformations on all steps in the pipeline and not just the estimator. If the gridsearch is not searching through the hyperparameters of the transformers, needless computations are being performed which may come with a larger runtime of the gridsearch.

## GridSearchCV inside the pipeline
`GridSearchCV` can be the final step of the pipeline. Since GridSearchCV is only operating on a estimator that is not a pipeline, the name of hyperparameters is simply the original name from the estimator. For example, to modify the regularization hyperparameter of the ridge estimator, the string to that refers to this hyperparameter is simply "alpha". The disadvantage of this approach is that you can only search through the hyperparameters of the estimator and not of any of the transformers. However, since the gridsearch only starts at the last step of the pipeline, the transformations of the dataset occurs once, which is computationally more efficient.

In [ ]:
# include gridsearch estimator inside of the pipeline

h_params = {'alpha': np.logspace(-3, 3, 20)}
gs = GridSearchCV(Ridge(), h_params)
pipe_2 = Pipeline([('scaler', StandardScaler()), ('gs_est', gs)])
pipe_2.fit(X_train, y_train)
print pipe_2.score(X_test, y_test)

## Using RandomizedSearchCV as an alternative to GridSearchCV

The number of gridpoints to explore when using `GridSearchCV` grows exponentially with the number of different types of hyperparameters you are exploring. You can consider each hyperparameter as a dimension in a n-dimensional space. An alternate to `GridSearhCV` is `RandomizedSearchCV` which constructs the grid of hyperparameters but randomly samples from the grid. The number of grid points sampled is controlled by the `n_iter` keyword. It has been shown that a randomized search can perform just as well as a full gridsearch but with only a fraction of the runtime. A "savings" of computational time can then be "invested" into exploring other hyperparameters or using a larger grid.

In [ ]:
# use randomized search
h_params = {'est__alpha': np.logspace(-4, 4, 100)}
gs_est = RandomizedSearchCV(pipe_1, h_params, cv=3, n_jobs=2, verbose=1, random_state=0, n_iter=20)
gs_est.fit(X_train, y_train)
print gs_est.score(X_test, y_test)

## Using dill to serialize estimators
`pickle` is a Python standard library that serializes Python objects. Serialization is the process of converting a data structure or object into a byte stream that can stored to disk. Deserialization refers to the opposite process where a byte stream is constructed to form a data structure or object. Think of pickling as a way to save Python objects to disk so they can be loaded and used at a later time. The `pickle` library has its limitations on what objects it can successfully serialize. Fortunately, the `dill` library improves on `pickle` by extending the set of possible Python data types that can be serialize. The syntax between `pickle` and `dill` are nearly identical.

It is useful to serialize `scikit-learn` estimators, especially after they have been trained. To serialize our trained estimator, we simply open a file and call `dill.dump` on the file object. Remember, it is good practice to use the `with` statement when dealing with file objects.

In [ ]:
import dill

with open('pipe_estimator.dill', 'w') as fp:
    dill.dump(pipe_1, fp)

In the above code, when using `open`, the `w` signafies that we want write access to the file. The file extension can be anything, however, using "dill" is descriptive and a nice convention. To deserialize the estimator, use `load` on the dill file. Note that `r` was used with `open` because we need read access to the file.

In [ ]:
with open('pipe_of_ridge.dill', 'r') as fp:
    pipe_1 = dill.load(fp)

Now we have the estimator loaded in memory and is ready to be used. If you are curious about the memory requirements of different trained estimators, you can use `dill` and check the resulting file size.

In [ ]:
print pipe_1.score(X_test, y_test)